In [2]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df = pd.read_csv('vgsales-12-4-2019.csv')
df.head(10)

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,nan,7.700,...,nan,nan,nan,nan,2006.000,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,nan,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,nan,10.000,...,nan,nan,nan,nan,1985.000,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,nan,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,nan,8.200,...,nan,nan,nan,nan,2008.000,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.700,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,nan,nan,...,nan,nan,nan,nan,2017.000,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,nan,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,nan,8.000,...,nan,nan,nan,nan,2009.000,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.800,/games/boxart/full_7295041AmericaFrontccc.jpg
5,6,Pokemon Red / Green / Blue Version,pokmon-red,Role-Playing,E,GB,Nintendo,Game Freak,nan,9.400,...,nan,nan,nan,nan,1998.000,NaN,http://www.vgchartz.com/game/4030/pokemon-red-...,1,nan,/games/boxart/full_6442337AmericaFrontccc.png
6,7,New Super Mario Bros.,new-super-mario-bros,Platform,E,DS,Nintendo,Nintendo EAD,nan,9.100,...,nan,nan,nan,nan,2006.000,NaN,http://www.vgchartz.com/game/1582/new-super-ma...,1,nan,/games/boxart/full_2916260AmericaFrontccc.jpg
7,8,Tetris,tetris,Puzzle,E,GB,Nintendo,Bullet Proof Software,nan,nan,...,nan,nan,nan,nan,1989.000,NaN,http://www.vgchartz.com/game/4534/tetris/?regi...,1,nan,/games/boxart/3740960ccc.jpg
8,9,New Super Mario Bros. Wii,new-super-mario-bros-wii,Platform,E,Wii,Nintendo,Nintendo EAD,nan,8.600,...,nan,nan,nan,nan,2009.000,NaN,http://www.vgchartz.com/game/35076/new-super-m...,1,9.100,/games/boxart/full_1410872AmericaFrontccc.jpg
9,10,Minecraft,minecraft,Misc,NaN,PC,Mojang,Mojang AB,nan,10.000,...,nan,nan,nan,nan,2010.000,05th Aug 18,http://www.vgchartz.com/game/47724/minecraft/?...,1,nan,/games/boxart/full_minecraft_1AmericaFront.png


In [4]:
df.columns

Index(['Rank', 'Name', 'basename', 'Genre', 'ESRB_Rating', 'Platform',
       'Publisher', 'Developer', 'VGChartz_Score', 'Critic_Score',
       'User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 'PAL_Sales',
       'JP_Sales', 'Other_Sales', 'Year', 'Last_Update', 'url', 'status',
       'Vgchartzscore', 'img_url'],
      dtype='object')

In [5]:
print(sum(df['ESRB_Rating'].notnull()))

23623


In [6]:
df = df[df['Global_Sales'].notna()]

In [7]:
df

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
19,20,Grand Theft Auto V,grand-theft-auto-v,Action,M,PS3,Rockstar Games,Rockstar North,nan,9.400,...,6.370,9.850,0.990,3.120,2013.000,NaN,http://www.vgchartz.com/game/65884/grand-theft...,1,nan,/games/boxart/full_6510540AmericaFrontccc.jpg
20,21,Grand Theft Auto V,grand-theft-auto-v,Action,M,PS4,Rockstar Games,Rockstar North,nan,9.700,...,6.060,9.710,0.600,3.020,2014.000,03rd Jan 18,http://www.vgchartz.com/game/83196/grand-theft...,1,nan,/games/boxart/full_5563178AmericaFrontccc.jpg
30,31,Grand Theft Auto: Vice City,grand-theft-auto-vice-city,Action,M,PS2,Rockstar Games,Rockstar North,nan,9.600,...,8.410,5.490,0.470,1.780,2002.000,NaN,http://www.vgchartz.com/game/896/grand-theft-a...,1,nan,/games/boxart/827563ccc.jpg
32,33,Grand Theft Auto V,grand-theft-auto-v,Action,M,X360,Rockstar Games,Rockstar North,nan,nan,...,9.060,5.330,0.060,1.420,2013.000,NaN,http://www.vgchartz.com/game/65883/grand-theft...,1,nan,/games/boxart/full_9218923AmericaFrontccc.jpg
34,35,Call of Duty: Black Ops 3,call-of-duty-black-ops-3,Shooter,M,PS4,Activision,Treyarch,nan,nan,...,6.180,6.050,0.410,2.440,2015.000,14th Jan 18,http://www.vgchartz.com/game/85359/call-of-dut...,1,nan,/games/boxart/full_4990510AmericaFrontccc.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21237,21238,"Mind, Body & Soul: Blend It!","body & soul blend it""",Puzzle,NaN,DS,505 Games,Crush Digital,nan,nan,...,nan,0.000,nan,nan,2010.000,NaN,http://www.vgchartz.com/game/31394/mind-body-a...,1,nan,/games/boxart/full_2998242PALFrontccc.jpg
21238,21239,Ride! Carnival Tycoon,ride-carnival-tycoon,Strategy,E,PC,ValuSoft,ValuSoft,nan,nan,...,nan,0.000,nan,nan,2007.000,NaN,http://www.vgchartz.com/game/40441/ride-carniv...,1,nan,/games/boxart/full_756013AmericaFrontccc.jpg
21239,21240,Shinobido: Tales of the Ninja,shinobido-tales-of-the-ninja,Action,NaN,PSP,Sony Computer Entertainment,Acquire,nan,nan,...,nan,0.000,nan,nan,2007.000,NaN,http://www.vgchartz.com/game/20018/shinobido-t...,1,nan,/games/boxart/full_2848967PALFrontccc.jpg
21240,21241,Tactical Ops: Assault on Terror,tactical-ops-assault-on-terror,Shooter,NaN,PC,Atari,Kamehan Studios,nan,nan,...,nan,0.000,nan,nan,2002.000,NaN,http://www.vgchartz.com/game/36164/tactical-op...,1,nan,/games/boxart/full_942613AmericaFrontccc.jpg


In [8]:
df.to_csv("games_df.csv")